In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [15]:
# Define paths
train_data_dir = "D:/IIT Docs/2nd Year/Semester 1/CM 2603 - Data Science Project/Data/Dataset/Paddy"
img_height, img_width = 480, 640
batch_size = 32

In [16]:
# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 2568 images belonging to 3 classes.


In [18]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 640 images belonging to 3 classes.


In [19]:
# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29752\2156671516.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


In [20]:
# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [21]:
# Combine base model with custom classification head
model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples,
    epochs=5
)

Epoch 1/5
  81/2568 ━━━━━━━━━━━━━━━━━━━━ 2:09:03 3s/step - accuracy: 0.6497 - loss: 0.9347

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2568/2568 ━━━━━━━━━━━━━━━━━━━━ 333s 122ms/step - accuracy: 0.7280 - loss: 0.7029 - val_accuracy: 0.7875 - val_loss: 0.4731
Epoch 2/5
2568/2568 ━━━━━━━━━━━━━━━━━━━━ 300s 112ms/step - accuracy: 0.8292 - loss: 0.4308 - val_accuracy: 0.8188 - val_loss: 0.4393
Epoch 3/5
2568/2568 ━━━━━━━━━━━━━━━━━━━━ 305s 114ms/step - accuracy: 0.8630 - loss: 0.3656 - val_accuracy: 0.8297 - val_loss: 0.3941
Epoch 4/5
2568/2568 ━━━━━━━━━━━━━━━━━━━━ 302s 113ms/step - accuracy: 0.8771 - loss: 0.3299 - val_accuracy: 0.8734 - val_loss: 0.3151
Epoch 5/5
2568/2568 ━━━━━━━━━━━━━━━━━━━━ 303s 113ms/step - accuracy: 0.8972 - loss: 0.2770 - val_accuracy: 0.9156 - val_loss: 0.2454


In [25]:
# Save the model
model.save('paddy_model.h5')

In [ ]:
from keras.src.saving import load_model

model = load_model('paddy_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("paddy_model.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbfflcleg\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbfflcleg\assets
